In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from matplotlib.ticker import StrMethodFormatter
from matplotlib import rcParams
from matplotlib.pyplot import figure

In [6]:
social_df = pd.read_excel('../../data/social_criteria.xlsx').dropna()
env_criteria = pd.read_excel('../../data/env_cri.xlsx', sheet_name="Form Responses 1").dropna()
level_one_df = pd.read_excel('../../data/lvl1_cri_pairwise.xlsx') 

In [13]:
def map_letter_vals(cell_value):
    map_values = {'a': 9, 'b': 6.34, 'c': 3.67, 'd': 1, 'e': 0}
    if isinstance(cell_value, str):
        letter = cell_value.split('.')[0]
        return map_values[letter]
    return cell_value

In [14]:
social = social_df.copy()
env = env_criteria.copy()

In [15]:
social = social.applymap(map_letter_vals)
env = env.applymap(map_letter_vals)

In [16]:
display(social)

,recreation,Health,Property,Job
0,9.00,9.00,6.34,3.67
1,9.00,9.00,9.00,9.00
2,9.00,9.00,9.00,9.00
3,6.34,6.34,9.00,6.34
4,9.00,6.34,6.34,6.34
...,...,...,...,...
93,9.00,9.00,6.34,9.00
94,9.00,9.00,6.34,9.00
95,6.34,9.00,3.67,6.34
96,6.34,6.34,9.00,6.34


In [17]:
display(env)

,energy,storm,air
0,1.00,6.34,3.67
1,9.00,9.00,9.00
2,9.00,9.00,9.00
3,9.00,9.00,9.00
4,6.34,6.34,3.67
...,...,...,...
93,9.00,9.00,9.00
94,9.00,9.00,9.00
95,6.34,6.34,9.00
96,9.00,9.00,6.34


In [18]:
mat = np.zeros((4, 4))
mat_env = np.zeros((2, 2))

In [153]:
import numpy as np

# Convert DataFrame to NumPy array
social_np = social.to_numpy()

# Dictionary to hold the r values
r_values = {f'r{i}{j}': [] for i in range(2) for j in range(2) if i != j}

# Generate the r values
for x in range(len(social_np)):
    row = social_np[x, :]
    mat = np.divide.outer(row, row)

    for i in range(2):
        for j in range(2):
            if i != j:
                key = f'r{i}{j}'
                r_values[key].append(mat[i, j])


/var/folders/zh/4wmfpks542d6lsg3xw_k0ysh0000gn/T/ipykernel_31311/548265151.py:12: RuntimeWarning: divide by zero encountered in divide
  mat = np.divide.outer(row, row)
/var/folders/zh/4wmfpks542d6lsg3xw_k0ysh0000gn/T/ipykernel_31311/548265151.py:12: RuntimeWarning: invalid value encountered in divide
  mat = np.divide.outer(row, row)


In [112]:
import numpy as np

# Convert DataFrame to NumPy array
env_np = env.to_numpy()

# List to hold the e_r10 values
e_r10 = []

# Generate the e_r10 values
for x in range(len(env_np)):
    row = env_np[x, :]
    mat_env = np.divide.outer(row, row)

    # Append the necessary value to e_r10
    e_r10.append(mat_env[1, 0])

/var/folders/zh/4wmfpks542d6lsg3xw_k0ysh0000gn/T/ipykernel_31311/2355901908.py:12: RuntimeWarning: divide by zero encountered in divide
  mat_env = np.divide.outer(row, row)
/var/folders/zh/4wmfpks542d6lsg3xw_k0ysh0000gn/T/ipykernel_31311/2355901908.py:12: RuntimeWarning: invalid value encountered in divide
  mat_env = np.divide.outer(row, row)


In [99]:
sample_data = [[4684956, 294200000, 28000000, 77123000],
               [34016111, 420900000, 60000000, 154246000],
               [51271313, 547500000, 93000000, 231369000],
               [62912556, 674200000, 121000000, 308492000]]
sample_data = np.array(sample_data)

In [100]:
sample_data_env = [[6713580, 9820003],
                   [16307399, 22391744],
                   [22785416, 31204186],
                   [28405151, 39210732]]
sample_data_env = np.array(sample_data_env)

In [125]:
sample_data = [[29.84, 1110],
               [149.226, 1110]]
sample_data = np.array(sample_data)

In [126]:
sample_data_env = [[25, 1627, 193],
                   [40, 2637, 313]]
sample_data_env = np.array(sample_data_env)

In [127]:
data_matrices = [sample_data, sample_data_env]

In [128]:
def create_pairwise(column, arr):
    col_values = arr[:, column]
    col_reshaped = col_values.reshape(-1, 1)
    mat = col_reshaped / col_values
    return np.round(mat, 4)

def prio_vec(A):
    e = np.linalg.eig(A)[1][:, 0]
    p = e / e.sum()

    return p.real

In [130]:
local_vars = []
for matrix in data_matrices:
    pairwise_mat = np.zeros(matrix.shape)
    for col_idx in range(matrix.shape[1]):
        col = matrix[:, col_idx]
        min_val = col.min()
        range_val = (matrix[:, col_idx].max() - min_val) / matrix.shape[0]

        # Check if range_val is zero to avoid division by zero
        if range_val != 0:
            pairwise_mat[:, col_idx] = 1 + (matrix[:, col_idx] - min_val) / range_val
        else:
            # Handle the case when range_val is zero
            # For example, you can assign a default value or skip
            pairwise_mat[:, col_idx] = 1  # This is a placeholder, adjust as needed

    pairwise_matrices = [create_pairwise(i, pairwise_mat) for i in range(matrix.shape[1])]
    local_vars.append(np.array([prio_vec(mat) for mat in pairwise_matrices]).T)


In [131]:
local_var_combined = np.concatenate(local_vars, axis=1)

In [158]:
wts_25 = []
wts_50 = []
wts_75 = []
wts_100 = []

level_one_cri = pd.read_excel('../../data/lvl1_cri_pairwise.xlsx')
level_one_cri = level_one_cri.iloc[:2, 1:3]

import numpy as np
import random

def simulator(n, r_values, level_one_cri, local_var_combined, wts_25, wts_50, wts_75, wts_100):
    count = 0
    s_mat = np.ones((2, 2))
    e_mat = np.ones((3, 3))

    for _ in range(n):
        try:
            # Fill s_mat with random choices from r_values
            for k, v in r_values.items():
                i, j = map(int, k[1:])
                s_mat[i, j] = random.choice(v)
                s_mat[j, i] = 1 / s_mat[i, j]  # Inverse for the symmetric element
    
            # Similar logic for e_mat if you have e_r values
            try:
                e_mat[1][0] = random.choice(e_r10)
                e_mat[0][1] = 1 / e_mat[1][0]
            except:
                pass
    
            # Eigenvalue calculations for s_mat
            w1, v1 = np.linalg.eig(s_mat)
            lambda_max1 = max(w1)
            CI1 = (lambda_max1 - len(s_mat)) / (len(s_mat) - 1)
            CR1 = CI1 / 0.90
    
            w2, v2 = np.linalg.eig(e_mat)
            lambda_max2 = max(w2)
            CI2 = (lambda_max2 - len(e_mat)) / (len(e_mat) - 1)
            CR2 = CI2 / 0.90
    
            if CR1 > 0.1 or abs(prio_vec(s_mat)[0]) > 1 or CR2 > 0.1 or abs(prio_vec(e_mat)[0]) > 1:
                continue
            else:
                mat1 = (prio_vec(level_one_cri)[1] * prio_vec(s_mat))
                mat2 = (prio_vec(level_one_cri)[0] * prio_vec(e_mat))
    
                multply_mat = np.concatenate((mat1, mat2))
    
                count += 1
                wts = (np.dot(local_var_combined, multply_mat))
                wts_25.append(wts[0])
                wts_50.append(wts[1])
        except:
            pass

    return countwts_25 = []
wts_50 = []
wts_75 = []
wts_100 = []

level_one_cri = pd.read_excel('../../data/lvl1_cri_pairwise.xlsx')
level_one_cri = level_one_cri.iloc[:2, 1:3]

import numpy as np
import random

def simulator(n, r_values, level_one_cri, local_var_combined, wts_25, wts_50, wts_75, wts_100):
    count = 0
    s_mat = np.ones((2, 2))
    e_mat = np.ones((3, 3))

    for _ in range(n):
        try:
            # Fill s_mat with random choices from r_values
            for k, v in r_values.items():
                i, j = map(int, k[1:])
                s_mat[i, j] = random.choice(v)
                s_mat[j, i] = 1 / s_mat[i, j]  # Inverse for the symmetric element
    
            # Similar logic for e_mat if you have e_r values
            try:
                e_mat[1][0] = random.choice(e_r10)
                e_mat[0][1] = 1 / e_mat[1][0]
            except:
                pass
    
            # Eigenvalue calculations for s_mat
            w1, v1 = np.linalg.eig(s_mat)
            lambda_max1 = max(w1)
            CI1 = (lambda_max1 - len(s_mat)) / (len(s_mat) - 1)
            CR1 = CI1 / 0.90
    
            w2, v2 = np.linalg.eig(e_mat)
            lambda_max2 = max(w2)
            CI2 = (lambda_max2 - len(e_mat)) / (len(e_mat) - 1)
            CR2 = CI2 / 0.90
    
            if CR1 > 0.1 or abs(prio_vec(s_mat)[0]) > 1 or CR2 > 0.1 or abs(prio_vec(e_mat)[0]) > 1:
                continue
            else:
                mat1 = (prio_vec(level_one_cri)[1] * prio_vec(s_mat))
                mat2 = (prio_vec(level_one_cri)[0] * prio_vec(e_mat))
    
                multply_mat = np.concatenate((mat1, mat2))
    
                count += 1
                wts = (np.dot(local_var_combined, multply_mat))
                wts_25.append(wts[0])
                wts_50.append(wts[1])
        except:
            pass

    return count

In [159]:
simulator(5000, r_values, level_one_cri, local_var_combined, wts_25, wts_50, wts_75, wts_100)

/var/folders/zh/4wmfpks542d6lsg3xw_k0ysh0000gn/T/ipykernel_31311/3122386164.py:23: RuntimeWarning: divide by zero encountered in scalar divide
  s_mat[j, i] = 1 / s_mat[i, j]  # Inverse for the symmetric element
/var/folders/zh/4wmfpks542d6lsg3xw_k0ysh0000gn/T/ipykernel_31311/3122386164.py:28: RuntimeWarning: divide by zero encountered in scalar divide
  e_mat[0][1] = 1 / e_mat[1][0]


4549

In [160]:
for k, v in r_values.items():
    print(list(map(int, k[1:])))

[0, 1]
[1, 0]


In [161]:
print(len(wts_25))

4549


In [165]:
column_names = ["wts_25", "wts_50", "wts_75", "wts_100"]

wts = pd.DataFrame(columns=column_names)

wts['wts_25'] = wts_25
wts['wts_50'] = wts_50
wts['wts_75'] = wts_25
wts['wts_100'] = wts_50

wts = wts.round(5)

In [170]:
print(pd.unique(wts['wts_25']))

[0.30555 0.31518 0.29591 0.29073 0.33731 0.28218 0.34597 0.32892 0.27378
 0.34999 0.32037]


In [169]:
import streamlit as st
st.header('Range of weights for all four alternatives')
figure(figsize=(6.9, 3))

rcParams['font.family'] = 'Arial'

plt.hist(x=wts['wts_25'], bins=len(pd.unique(wts['wts_25'])), color='black',
         alpha=1)
plt.hist(x=wts['wts_50'], bins=len(pd.unique(wts['wts_50'])), color='brown',
         alpha=1)
plt.hist(x=wts['wts_75'], bins=len(pd.unique(wts['wts_75'])), color='blue',
         alpha=1)
plt.hist(x=wts['wts_100'], bins=len(pd.unique(wts['wts_100'])), color='green',
         alpha=1)
plt.gcf().set_size_inches(7.5, 3)

# plt.xticks(np.linspace(0.0, 0.45, 10), rotation = 0)


plt.legend(['25% GI', '50% GI', '75% GI', '100% GI'], fontsize=7)
plt.xticks(fontsize=6)
plt.yticks(fontsize=6)
plt.ylabel('Density', fontsize=7)
plt.xlabel('Weight', fontsize=7)

plt.gca().xaxis.set_major_formatter(StrMethodFormatter('{x:,.3f}'))

txt = "(A). Range of weights for all four alternatives"
plt.figtext(0.5, -0.028, txt, wrap=True, horizontalalignment='center', fontsize=7)

plt.savefig('wts.png', dpi=300, bbox_inches="tight")

st.pyplot(plt)

plt.hist(x=wts['wts_25'], bins=len(pd.unique(wts['wts_25'])), color='black',
         alpha=1)

# plt.legend(['25% GI'])
plt.ylabel('Density', fontsize=7)
plt.xlabel('Weight', fontsize=7)

from matplotlib.ticker import StrMethodFormatter
st.header('Range of weights for 25% GI')
plt.gca().xaxis.set_major_formatter(StrMethodFormatter('{x:,.3f}'))

fig = plt.gcf()
fig.set_size_inches(3.25, 2.3)

plt.xticks(fontsize=6)
plt.yticks(fontsize=6)

txt = "(B) 25% GI, 75% TI"
plt.figtext(0.54, -0.02, txt, wrap=True, horizontalalignment='center', fontsize=7)

plt.tight_layout()

plt.savefig('wts_25.png', dpi=300, bbox_inches="tight")

st.pyplot(plt)

st.header('Range of weights for 50% GI')
plt.hist(x=wts['wts_50'], bins=len(pd.unique(wts['wts_50'])), color='brown',
         alpha=1)

# plt.legend(['50% GI'])
plt.ylabel('Density', fontsize=7)
plt.xlabel('Weight', fontsize=7)

from matplotlib.ticker import StrMethodFormatter

plt.gca().xaxis.set_major_formatter(StrMethodFormatter('{x:,.3f}'))

fig = plt.gcf()
fig.set_size_inches(3.25, 2.3)

plt.xticks(fontsize=6)
plt.yticks(fontsize=6)

txt = "(C) 50% GI, 50% TI"
plt.figtext(0.54, -0.02, txt, wrap=True, horizontalalignment='center', fontsize=7)

plt.tight_layout()
plt.savefig('wts_50.png', dpi=300, bbox_inches="tight")

st.pyplot(plt)

st.header('Range of weights for 75% GI')
plt.hist(x=wts['wts_75'], bins=len(pd.unique(wts['wts_75'])), color='blue',
         alpha=1)

# plt.legend(['75% GI'])
plt.ylabel('Density', fontsize=7)
plt.xlabel('Weight', fontsize=7)

from matplotlib.ticker import StrMethodFormatter

plt.gca().xaxis.set_major_formatter(StrMethodFormatter('{x:,.3f}'))

fig = plt.gcf()
fig.set_size_inches(3.25, 2.3)

plt.xticks(fontsize=6)
plt.yticks(fontsize=6)

txt = "(D) 75% GI, 25% TI"
plt.figtext(0.54, -0.02, txt, wrap=True, horizontalalignment='center', fontsize=7)

plt.tight_layout()
plt.savefig('wts_75.png', dpi=300, bbox_inches="tight")

st.pyplot(plt)

st.header('Range of weights for 100% GI')
plt.hist(x=wts['wts_100'], bins=len(pd.unique(wts['wts_100'])), color='green',
         alpha=1)

# plt.legend(['100% GI'])
plt.ylabel('Density', fontsize=7)
plt.xlabel('Weight', fontsize=7)

from matplotlib.ticker import StrMethodFormatter

plt.gca().xaxis.set_major_formatter(StrMethodFormatter('{x:,.3f}'))

fig = plt.gcf()
fig.set_size_inches(3.25, 2.3)

plt.xticks(fontsize=6)
plt.yticks(fontsize=6)

txt = "(E) 100% GI"
plt.figtext(0.54, -0.02, txt, wrap=True, horizontalalignment='center', fontsize=7)
plt.tight_layout()
plt.savefig('wts_100.png', dpi=300, bbox_inches="tight")

st.pyplot(plt)

from matplotlib import font_manager

font_manager_inst = font_manager.fontManager

import matplotlib.pyplot as plt
import numpy

from matplotlib import rcParams

rcParams['font.family'] = 'Arial'

data = r10
#
sorted_random_data = numpy.sort(data)
p = 1. * numpy.arange(len(sorted_random_data)) / float(len(sorted_random_data) - 1)
#

fig = plt.figure()
# fig.suptitle('CDF of R[1,0]')
ax2 = fig.add_subplot(111)

ax2.plot(sorted_random_data, p)
ax2.set_xlabel("(A) " + '$c_2$/$c_1$', fontsize=22)
ax2.set_ylabel('Cumulative probabilty', fontsize=22)
plt.yticks(fontsize=20)
plt.xticks(fontsize=20)
plt.savefig("1.png", dpi=300, bbox_inches="tight")
st.header("CDF of $c_2$/$c_1$")
st.pyplot(plt)


rcParams['font.family'] = 'Arial'

data = r20
#
sorted_random_data = numpy.sort(data)
p = 1. * numpy.arange(len(sorted_random_data)) / float(len(sorted_random_data) - 1)
#

fig = plt.figure()
# fig.suptitle('CDF of data points')
ax2 = fig.add_subplot(111)
ax2.plot(sorted_random_data, p)
ax2.set_xlabel("(B) " + '$c_3$/$c_1$', fontsize=22)
ax2.set_ylabel('Cumulative probabilty', fontsize=22)
plt.yticks(fontsize=20)
plt.xticks(fontsize=20)
plt.savefig("2.png", dpi=300, bbox_inches="tight")
st.header("CDF of $c_3$/$c_1$")
st.pyplot(plt)

rcParams['font.family'] = 'Arial'

data = r31
#
sorted_random_data = numpy.sort(data)
p = 1. * numpy.arange(len(sorted_random_data)) / float(len(sorted_random_data) - 1)
#

fig = plt.figure()
# fig.suptitle('CDF of data points')
ax2 = fig.add_subplot(111)
ax2.plot(sorted_random_data, p)
ax2.set_xlabel("(E) " + '$c_4$/$c_2$', fontsize=22)
ax2.set_ylabel('Cumulative probabilty', fontsize=22)
plt.yticks(fontsize=20)
plt.xticks(fontsize=20)
plt.savefig("5.png", dpi=300, bbox_inches="tight")
st.header("CDF of $c_4$/$c_2$")
st.pyplot(plt)


/var/folders/zh/4wmfpks542d6lsg3xw_k0ysh0000gn/T/ipykernel_31311/3763382187.py:82: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/var/folders/zh/4wmfpks542d6lsg3xw_k0ysh0000gn/T/ipykernel_31311/3763382187.py:108: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/var/folders/zh/4wmfpks542d6lsg3xw_k0ysh0000gn/T/ipykernel_31311/3763382187.py:133: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


DeltaGenerator()